#### Notebook 1 - Mobile traffic data
Streaming of high hype sport events (particularly football matches) is creating some additional stress on network capacity.
Topic became even more of interest when a pure internet broadcaster acquired the right for serie A matches for next season. 
The project is about some numerical investigation on the relationship between traffic growth on a mobile network in Italy and the scheduling of football matches involving italian teams. Specifically, investigation will address the following questions:

- Can we identify a significative numerical correlation between mobile traffic peaks and footbal matches schedule?
- If yes, can we add insigth for classes of football event parameters showing higher correlations (e.g. number of contemporary matches, hype of a single match, types of broadcaster)? 
- Can the information above be used to tune traffic growth forecasts on a mobile network?

The activity is intended as an investigation exercise, so data actually used, though obtained from real measurement, will first be normalized to range between artificially set bounds.   

#### Datasets

The notebook will leverage on following data:

- Some real traffic measurement from a mobile network on a generic geographical area in Italy over about 2 years. Data are normalized in a way that reduces all used kpi to range between 1 and 2. Only normalized and region wide values are shown.
- Scheduling of major footbal competitions involving italian teams (I considered 'Serie A' and 'Champions League')
- Information about the broadcaster distribuiting the matches (internet only or pay tv+internet)

#### Notebook organization
Activity is organized into different jupyter notebooks:
- The first notebook (the present one) is about preparing reading traffic data and adding normalization. As actual starting data will not be shared, this notebook is only intended for reference
- The second notebook will analize and prepare normalized data for actual usage 
- Third notebook reads, manipulates and prepare sport events dataframe
- Fourth notebook  merges all collected and preprocessed data and investigate relationship to try to answer the target questions

In [286]:
import pandas as pd

In [287]:
cd "C:\\Users\\rpaglin\\OneDrive - Vodafone Group\\Desktop\\Nanodegree prj1 data"

C:\Users\rpaglin\OneDrive - Vodafone Group\Desktop\Nanodegree prj1 data


In [302]:
def readxpottercsvfiles(path, filenames):
    ''' The function read a list of csv file with known format into a dataframe. Each CSV will consiste of 6 columns, 
    5 of them mainatined in the df:
    'DATE': date in yyyy-mm-dd format
    'TIME': Time of measurement as hh:mm:ss. Only hours are available in the used datafiles, so min and sec wil be dropped  
    'RESOURCE NAME': Network element that collected the measurement
    'KPI NAME': type of measurement
    'VALUE': actual measurement
    INPUT
    path: directory to be searched for files
    filenames: python list of filenames
    OUTPUT
    The function return a dataframe consolidating info from all files read.
    DATE field is translated into a datetime, while TIME field is memorized with an integer representing hour  
    '''
    df = pd.read_csv(path+filenames[0],thousands=',',usecols=['RESOURCE NAME','KPI NAME', 'DATE', 'TIME','VALUE'])
    for fn in filenames[1:]:
        df_tmp = pd.read_csv(path+fn, thousands=',',usecols=['RESOURCE NAME','KPI NAME', 'DATE', 'TIME','VALUE'])
        df=pd.concat([df,df_tmp])
    df.drop_duplicates(['DATE','TIME','KPI NAME','RESOURCE NAME'], inplace=True)
    df['DATE']=df['DATE'].apply(lambda x: pd.datetime(int(x[0:4]),int(x[5:7]),int(x[8:10])))
    df['TIME']=df['TIME'].apply(lambda x: int(x[0:2]))
    return df

In [303]:
def normalize_values(df, col, w_0,w_m):
    ''' Purpose of this function is to partially hyde actual network data, forcing measurements 
    in the analized period to range from artificially set min and max values.
    As the purpose of the activity is to exercise on a methodology, in the follwoing we will consider 
     normalized data as valid values and we will not bother on the effect of this normalization on results
     
    INPUT
    df: a dataframe containing a column to be normalized
    col: the column identifier
    w_0: Artificial value that will be assigned to the max measurement taken in day 1
    w_m: Artificial value that will be assigned to the max measurement taken in last full day
    OUTPUT
    The function return df dataframe with the column 'col' normalized around w_0 and w_m
    Note that the approach taken will not force outlyers to be in the expected range
    '''
    df1=df.groupby('DATE', axis=0).max()
    v_m=df1[col].iloc[-2] #max on last full day
    v_0=df1[col].iloc[0]  #max on first full day
    delta_v=v_m-v_0+0.
    delta_w=w_m-w_0+0.
    delta_ratio=delta_w/delta_v
    df[col]=w_m-(v_m-df[col])*delta_ratio
    return df

#### Throughput Data

In [290]:
path='C:\\Users\\rpaglin\\OneDrive - Vodafone Group\\Desktop\\Nanodegree prj1 data\\Xpotter Thr\\'

In [291]:
filenames=['report_rpaglin_thr19Q1.csv','report_rpaglin_thr19Q2.csv','report_rpaglin_thr19Q3.csv','report_rpaglin_thr19Q4.csv']
filenames+=['report_rpaglin_thr20Q1.csv','report_rpaglin_thr20Q2.csv','report_rpaglin_thr20Q3.csv']
filenames+=['report_rpaglin_thr20Q4.csv','report_rpaglin_thr21Q1.csv','report_rpaglin_thr21Q2.csv']

In [292]:
thr_df=readxpottercsvfiles(path, filenames)

In [293]:
thr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397153 entries, 0 to 27607
Data columns (total 5 columns):
RESOURCE NAME    397153 non-null object
KPI NAME         397153 non-null object
DATE             397153 non-null datetime64[ns]
TIME             397153 non-null int64
VALUE            395291 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 18.2+ MB


In [294]:
thr_df=thr_df.groupby(by=['DATE', 'TIME'], axis=0).sum()

In [295]:
thr_df=normalize_values(thr_df,'VALUE',1.,2.)

In [296]:
thr_df.groupby('DATE').max().head()

,VALUE
DATE,
2019-01-01,1.000000
2019-01-02,0.995058
2019-01-03,0.995295
2019-01-04,0.967564
2019-01-05,0.925946


#### Connection data

In [297]:
path='C:\\Users\\rpaglin\\OneDrive - Vodafone Group\\Desktop\\Nanodegree prj1 data\\Xpotter Cn\\'

In [298]:
filenames=['report_rpaglin_cn1_19Q2.csv','report_rpaglin_cn1_19Q3.csv','report_rpaglin_cn1_19Q4.csv']
filenames+=['report_rpaglin_cn1_20Q1.csv','report_rpaglin_cn1_20Q2.csv','report_rpaglin_cn1_20Q3.csv']
filenames+=['report_rpaglin_cn1_20Q4.csv','report_rpaglin_cn1_21Q1.csv','report_rpaglin_cn1_21Q2.csv']
filenames+=['report_rpaglin_cn2_19Q1.csv']
filenames+=['report_rpaglin_cn2_19Q2.csv','report_rpaglin_cn2_19Q3.csv','report_rpaglin_cn2_19Q4.csv']
filenames+=['report_rpaglin_cn2_20Q1.csv','report_rpaglin_cn2_20Q2.csv','report_rpaglin_cn2_20Q3.csv']
filenames+=['report_rpaglin_cn2_20Q4.csv','report_rpaglin_cn2_21Q1.csv','report_rpaglin_cn2_21Q2.csv']


In [299]:
cn_df=readxpottercsvfiles(path, filenames)
cn_df=cn_df.groupby(by=['DATE', 'TIME'], axis=0).sum()
cn_df=normalize_values(cn_df,'VALUE',1.,2.)
cn_df.head(3)

VALUE
DATE       TIME          
2019-01-01 0     0.949642
           1     0.920442
           2     0.885188

#### Merging info into a single dataframe and saving

In [300]:
thr_df.rename(columns={'VALUE':'Thr (Norm)'},inplace=True)
cn_df.rename(columns={'VALUE':'Conn (Norm)'},inplace=True)
nw_df=pd.concat([thr_df, cn_df], axis=1)
nw_df.reset_index(inplace=True)
nw_df.info()
nw_df.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21036 entries, 0 to 21035
Data columns (total 4 columns):
DATE           21036 non-null datetime64[ns]
TIME           21036 non-null int64
Thr (Norm)     20923 non-null float64
Conn (Norm)    21036 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 657.5 KB


,DATE,TIME,Thr (Norm),Conn (Norm)
0,2019-01-01,0,0.951451,0.949642
1,2019-01-01,1,0.839571,0.920442
2,2019-01-01,2,0.710031,0.885188
3,2019-01-01,3,0.589674,0.858900
4,2019-01-01,4,0.497652,0.844702


In [301]:
nw_df.to_csv('C:\\Users\\rpaglin\\OneDrive - Vodafone Group\\Desktop\\Nanodegree prj1 data\\nw_df.csv', index=False)
